##SETUP

In [1]:
import os
import time
import timeit
import pandas as pd
import random
import numpy as np
# Android ADB
from scrcpy import Client, const
from subprocess import check_output,Popen
import threading

# Image processing
from PIL import Image
import cv2
import pytesseract
# Notebook
from IPython.display import display,clear_output
from tqdm.notebook import trange, tqdm
# internal
import bot_core
import bot_perception

import importlib
importlib.reload(bot_core)
importlib.reload(bot_perception)


SLEEP_DELAY=0.1

# Start Scrcpy (once after PC restart)
start_scrcpy=False
if start_scrcpy:
    proc = Popen(['C:\Programs\Scrcpy\scrcpy'], shell=True)
    time.sleep(6) # <-- sleep for 6''
    proc.terminate() # <-- terminate the process


bot = bot_core.Bot()

In [2]:
bot.client.control.text("test")


b'\x01\x00\x00\x00\x04test'

## Game stats

In [159]:
# get stored mana 
#bot.getText(220,1360,90,50,new=False,digits=True)

# get unit mana 
bot.getText(450,1360,90,50,new=False,digits=True)

# Get upgrade mana
#bot.getText(20,1540,700,50,new=False,digits=True).split('6')


'70'

In [3]:
names=bot.scan_grid()
grid_df=bot_perception.grid_status(names)
grid_df=bot_perception.grid_status(names,grid_df)
display(grid_df)
prev_grid= grid_df.copy()
#cache_grid= grid_df.copy()

,grid_id,unit,probability,rank,rank_error,position,Age
0,icon_0,empty.png,999.0,0,0.0,"[0, 0]",1.0
1,icon_1,alchemist.png,700.0,1,16184.5,"[0, 1]",1.0
2,icon_2,sharpshooter.png,335.0,2,940.0,"[0, 2]",1.0
3,icon_3,alchemist.png,228.0,1,1577.5,"[0, 3]",1.0
4,icon_4,vampire.png,187.0,1,1459.5,"[0, 4]",1.0
5,icon_5,empty.png,999.0,0,0.0,"[1, 0]",1.0
6,icon_6,empty.png,999.0,0,0.0,"[1, 1]",1.0
7,icon_7,vampire.png,179.0,1,4235.5,"[1, 2]",1.0
8,icon_8,lightning.png,190.0,1,2373.0,"[1, 3]",1.0
9,icon_9,lightning.png,178.0,1,1476.5,"[1, 4]",1.0


In [709]:
df_split,unit_series, df_groups, group_keys=bot_core.grid_meta_info(grid_df)
group_keys.remove(('empty.png',0))
#unit_series.drop(labels='empty.png',level=0)
merge_series = unit_series[unit_series>2]
merge_target =  merge_series.sample().index[0]
merge_df=df_split.get_group(merge_target)
merge_df.sample(n=2)

,grid_id,unit,probability,rank,rank_error,position,Age
12,icon_12,priest.png,175.0,1,2068.5,"[2, 2]",0.0
13,icon_13,priest.png,200.0,1,1269.5,"[2, 3]",4.0


# Play Game

In [116]:
bot.getScreen()

In [3]:
bot.restart_RR()

## Extra unit logic 

In [41]:
##Extra unit logic 
# Try to merge any priestess
#if 'priest.png' in unit_list:
#    if df_groups['priest.png']<1:
#        bot.merge_unit(grid_df,'priest.png')

# Basic Loop

In [52]:
def try_merge(rank=1,prev_grid=None):
    info=''
    merge_df =None
    names=bot.scan_grid(new=True)
    grid_df=bot_perception.grid_status(names,prev_grid=prev_grid)
    df_split,unit_series, df_groups, group_keys=bot_core.grid_meta_info(grid_df)
    # Select stuff to merge
    merge_series = unit_series[unit_series>=2] # At least 2 units ## ADD MIME to every count, use sample rank and check if mime exist
    # check if grid full
    if ('empty.png',0) in group_keys:
        # Try to merge priest
        merge_priest = bot_core.filter_keys(merge_series,['priest.png'])
        if not merge_priest.empty:
            merge_df = bot.merge_unit(df_split,merge_priest)
        # Merge if full board
        if df_groups['empty.png']<=2:
            info='Merging!'
            # Add criteria
            merge_series = bot_core.filter_keys(merge_series,[rank,'priest.png'])
            if not merge_series.empty:
                # drop duplicated indices
                merge_series = merge_series[~merge_series.index.duplicated()]
                # Take index name of random sample
                merge_target =  merge_series.sample().index[0] 
                # Retrieve group    
                merge_df=df_split.get_group(merge_target)
                #merge_df=merge_df.sort_values(by='Age',ascending=False).reset_index(drop=True)
                # Send merge command
                merge_df = bot.merge_unit(df_split,merge_series)
            else:
                info='not enough filtered targets!'
        else: info= 'need more units!'
    # If grid seems full, merge any
    else:
        info = 'Full Grid - Merging!'
        merge_df = bot.merge_unit(df_split,merge_series)
    return grid_df,unit_series,merge_df,info
    
def get_button_pos(df,button):
    button=button+'.png'
    pos=df[df['icon']==button]['pos [X,Y]'].reset_index(drop=True)[0]
    return pos
# Mana level cards
def mana_level(cards, hero_power=False):
    upgrade_pos_dict={
    1:[100,1500], 2:[200,1500],
    3:[350,1500], 4:[500,1500],
    5:[650,1500] }
    # Level each card
    for card in cards:
        bot.click(*upgrade_pos_dict[card])
    if hero_power:
        bot.click(800,1500)

def battle_screen(start=False):
    # Scan screen for any key buttons
    button_df = bot.get_current_icons()
    # filter relevant buttons
    button_df = button_df[button_df['icon'].isin(['back_button.png','battle_icon.png','0cont_button.png','1quit.png', 'fighting.png','pvp_button.png'])]
    avail_buttons=button_df[button_df['available']==True].reset_index(drop=True)
    if not avail_buttons.empty:
        # list of buttons
        button_names=avail_buttons['icon'].tolist()
        if 'fighting.png' in button_names and not '0cont_button.png' in button_names:
            return avail_buttons,'fighting'
        # Start pvp if homescreen
        if start and 'pvp_button.png' in button_names and 'battle_icon.png' in button_names:
            pvp_pos = get_button_pos(button_df,'pvp_button')
            bot.click_button(pvp_pos)
            time.sleep(1)
            return avail_buttons, 'home'
        # Check first button is clickable
        first_button=button_names[0].split('.')[0]
        if first_button in ['back_button','battle_icon','0cont_button','1quit']:
            button_pos=avail_buttons['pos [X,Y]'].tolist()[0]
            bot.click_button(button_pos)
            return avail_buttons,'menu'
    else:
        bot.shell(f'input keyevent {const.KEYCODE_BACK}') #Force back
        return button_df,'lost'
        # add recursive call to restart last recorse?
battle_screen()

(              icon  available   pos [X,Y]
 0  back_button.png       True  (62, 1480),
 'menu')

In [129]:
#def play_dungeon(floor=5):
floor = 5
button_df = bot.get_current_icons()
avail_buttons=button_df[button_df['available']==True].reset_index(drop=True)

if (avail_buttons == 'dungeon_page.png').any(axis=None):
    print('Dungeon Page')
avail_buttons

for i in range(3):
    # Swipe down (change to swipe up for floor 10 cleared)
    [bot.swipe([2,0],[0,0]) for i in range(3)]
    bot.click(30,600) # stop scroll and scan screen for buttons
    button_df = bot.get_current_icons()
    avail_buttons=button_df[button_df['available']==True].reset_index(drop=True)
    # Look for correct chapter
    chapter_banner = avail_buttons[avail_buttons['icon'] == f'chapter_{int(np.ceil(floor/3))}.png']
    if not chapter_banner.empty: 
        pos = chapter_banner['pos [X,Y]'].iloc[0]
        # check button is near top of screen
        if pos[1] < 550:
            break
        continue
# Expand chapter
bot.click(pos[0]+450,pos[1])
# Click play floor only 1,2, 4,5, 7, 8 possible 
bot.click(pos[0],pos[1]+80+340*(floor%3))

    #button_df=button_df[button_df.isin({'icon':['back_button.png','battle_icon.png','0cont_button.png','1quit.png', 'fighting.png','pvp_button.png']})]
#420 pixel down offset for 1st floor in chapter
#760 pixel down offset for 2nd floor in chapter

#400-500 pixel offset right for expand chapter button
floor//3+1

KeyboardInterrupt: 

In [51]:
#button_df[button_df.isin(['back_button.png','battle_icon.png','0cont_button.png','1quit.png', 'fighting.png','pvp_button.png'])]
button_df = button_df[button_df['icon'].isin(['back_button.png','battle_icon.png','0cont_button.png','1quit.png', 'fighting.png','pvp_button.png'])]

TypeError: list indices must be integers or slices, not str

In [50]:
df_split,unit_series, df_groups, group_keys=bot_core.grid_meta_info(grid_df)
# Select stuff to merge
merge_series = unit_series[unit_series>=2] # At least 2 units ## ADD MIME to every count, use sample rank and check if mime exist
# check if grid full
if ('empty.png',0) in group_keys:
    # Try to merge priest
    merge_priest = bot_core.filter_keys(merge_series,['priest.png'])

In [13]:
df_split,unit_series, df_groups, group_keys=bot_core.grid_meta_info(grid_df)
# Select stuff to merge
merge_series = unit_series[unit_series>=2] # at least 2 units 
merge_series

unit           rank
lightning.png  1       3
alchemist.png  1       2
empty.png      0       2
priest.png     2       2
Name: unit, dtype: int64

## RUN BOT

In [9]:
wait=0
combat = 0
grid_df =None
# Main loop
while(1):
    output = battle_screen(start=True)
    if output[1]=='fighting':
        wait = 0
        combat+=1
        if combat>20:
            bot.restart_RR()
            combat = 0
        battle_bar=trange(8)
        for i in battle_bar:
                time.sleep(0.2)
                # Spawn units
                bot.click(450,1360)
                # Upgrade units
                mana_level([5,2,3,4],hero_power=True)
                # Try to merge units
                grid_df,unit_series,df_groups,info = try_merge(prev_grid=grid_df)
                clear_output()
                display([f'{combat}, {i+1}/8 '+output[1] +'  '+info,grid_df,unit_series[0:5],df_groups])
    else:
        combat=0
        wait+=1
        if wait>40:
            print('RESTARTING')
            bot.restart_RR()
            wait=0
        print(output[1],'wait count:',wait)

['19, 5/8 fighting  Merging!',
     grid_id             unit  probability  rank  rank_error position  Age
 0    icon_0       priest.png        258.0     3       327.0   [0, 0]  1.0
 1    icon_1    alchemist.png        318.0     3       217.5   [0, 1]  1.0
 2    icon_2      vampire.png        158.0     1      1193.5   [0, 2]  5.0
 3    icon_3        empty.png        999.0     0         0.0   [0, 3]  7.0
 4    icon_4      vampire.png        579.0     5       649.0   [0, 4]  0.0
 5    icon_5    alchemist.png        225.0     2       620.0   [1, 0]  0.0
 6    icon_6  executioner.png        256.0     1      2355.0   [1, 1]  2.0
 7    icon_7  executioner.png        164.0     1      1642.5   [1, 2]  1.0
 8    icon_8    alchemist.png        277.0     1      1757.0   [1, 3]  5.0
 9    icon_9    alchemist.png        248.0     2       180.0   [1, 4]  0.0
 10  icon_10    alchemist.png        339.0     2       535.0   [2, 0]  5.0
 11  icon_11    alchemist.png        273.0     1      1978.0   [2, 1]

KeyboardInterrupt: 